In [ ]:

import os, math, numpy as np, pandas as pd
from dataclasses import dataclass
from typing import List, Tuple

# ML
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression


# SQLAlchemy 
import pyodbc
from sqlalchemy import create_engine, text



2) SQL DE EXTRACCIÓN


In [16]:
import os
# Configuración 
server = os.environ.get('SQL_SERVER','')
database = os.environ.get('SQL_DB','')
username = os.environ.get('SQL_USER','')
password = os.environ.get('SQL_PASSWORD','')

# Crear cadena de conexión segura
params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},1433;"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)

engine_url = f"mssql+pyodbc:///?odbc_connect={params}"

# Crear conexión
engine = create_engine(engine_url, fast_executemany=True)

# Probar conexión
with engine.connect() as conn:
    ver = conn.execute(text("SELECT @@VERSION")).scalar()
    dbn = conn.execute(text("SELECT DB_NAME()")).scalar()
    print("Conectado a:", dbn)
    print("Servidor:", ver.splitlines()[0])

# Consulta base
SQL_EXTRACT = r"""
select
     [num_identificacion]
    ,[cod_pensum]
    ,[COD_PERIODO]
    ,[SEMESTRE]
    ,[ESTADO_ALUMNO]
    ,[Programa]
    ,[promedio]
    ,[CreditosAprobados]
    ,[CreditosTotales]
    ,[Creditos_Pendientes]
    ,[ULT_PER_PAGO]
    ,[Saldo]
    ,[Dupl]
    ,[Asignaturas_Aprobadas]
    ,[Asignaturas_Reprobadas]
    ,[Promedio_Per_Solicitado]
    ,case
        when [Status] = 'Retiro' AND ESTADOACTUAL = 'YA PAGO' THEN 'Posible Perdida Academica'
        when [Status] = 'Posible Egresado' AND ESTADOACTUAL = 'YA PAGO' THEN 'Posible Rematricula'
        else [Status] end as Status,
    NUEVO,
    UBICACION,
    sede,
    ESTADOACTUAL,
    Periodo_Paga
FROM
(
    SELECT
        a.[num_identificacion], a.[cod_pensum], a.[COD_PERIODO], a.[SEMESTRE],
        a.[ESTADO_ALUMNO], a.[Programa], a.[promedio],
        a.[CreditosAprobados], a.[CreditosTotales], a.[Creditos_Pendientes],
        a.[ULT_PER_PAGO], a.[Saldo], a.[Dupl],
        a.[Asignaturas_Aprobadas], a.[Asignaturas_Reprobadas], a.[Promedio_Per_Solicitado],
        a.[Status],
        t2.NUEVO, t2.UBICACION, t2.sede,
        case
            when t1.PERIODO is null then 'NO HA PAGADO'
            when t1.PERIODO = a.COD_PERIODO THEN 'NO HA PAGADO'
            else 'YA PAGO'
        end AS ESTADOACTUAL,
        case
            when t1.PERIODO is null then 'NO HA PAGADO'
            else t1.PERIODO
        end as Periodo_Paga
    from CUN_REPOSITORIO.GGALINDO.REMATRICULASNEW a
    left join (
        select DISTINCT Documento_Estudiante_zoho, max(PERIODO) as PERIODO
        from CUN_REPOSITORIO.dbo.Recibos_Contact2_Sin_Zoho
        where PERIODO in ('24V01','2024A','24V02','24V03','24P02','24V04','2024B','24V05','24V06','25v02','2025B','25v05','2025D')
          AND ESTADO_PAGO = 'PAGO'
          AND NCR_ANULACION = 0
        GROUP BY Documento_Estudiante_zoho
    ) t1 on a.num_identificacion = t1.Documento_Estudiante_zoho
    left join (
        select DISTINCT Documento_Estudiante_zoho, PERIODO, max(NUEVO) as NUEVO, UBICACION, sede
        from CUN_REPOSITORIO.dbo.Recibos_Contact2_Sin_Zoho
        where PERIODO in ('23V04','2023B','23V05','23V06','24V01','2024A','24V05','24V06','25v02','2025B')
          AND ESTADO_PAGO = 'PAGO'
          AND NCR_ANULACION = 0
        GROUP BY Documento_Estudiante_zoho, PERIODO, UBICACION, sede
    ) t2 on a.num_identificacion = t2.Documento_Estudiante_zoho
       and a.COD_PERIODO = t2.PERIODO
) BASE
"""

with engine.connect() as conn:
    df = pd.read_sql(SQL_EXTRACT, conn)

print("✅ Consulta ejecutada. Filas obtenidas:", len(df))


Conectado a: CUN_REPOSITORIO
Servidor: Microsoft SQL Server 2022 (RTM-GDR) (KB5035432) - 16.0.1115.1 (X64) 
✅ Consulta ejecutada. Filas obtenidas: 250346


3) CONFIG APP

In [17]:
@dataclass
class APP:
    col_id: str = "num_identificacion"
    col_periodo: str = "COD_PERIODO"
    col_periodo_paga: str = "Periodo_Paga"
    col_estado_actual: str = "ESTADOACTUAL"
    col_target: str = "y_rematricula"
    col_periodo_orden: str = "periodo_orden"
    seg_programa: str = "Programa"
    seg_sede: str = "sede"
    out_dir: str = r"C:\visual\rematricula_out"
    out_scoring: str = "scoring_valid.csv"
    topM: int = 1500

APP = APP()

# Periodos que aparecen en tus filtros (normalizados a MAYÚSCULA)
PERIODO_ORDEN = [
    "23V04","2023B","23V05","23V06",
    "24V01","2024A","24V02","24V03","24P02","24V04","2024B","24V05","24V06",
    "25V02","2025B","25V05","2025D"
]
ORDEN_MAP = {p.upper(): i for i, p in enumerate(PERIODO_ORDEN, start=1)}

4) UTILIDADES

In [18]:
def norm_periodo(s: pd.Series) -> pd.Series:
    return s.astype(str).str.upper().str.strip()

def build_gpu_xgb(y_train: pd.Series) -> XGBClassifier:
    pos = int((y_train == 1).sum())
    neg = int((y_train == 0).sum())
    spw = max(neg / max(pos, 1), 1.0)
    n_jobs = max((os.cpu_count() or 8) - 2, 4)
    return XGBClassifier(
        n_estimators=1200,
        max_depth=6,
        learning_rate=0.03,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=4,
        reg_lambda=1.2,
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        gpu_id=0,
        scale_pos_weight=spw,
        n_jobs=n_jobs,
    )

def infer_feature_sets(df: pd.DataFrame, target: str, drops: List[str]) -> Tuple[List[str], List[str], List[str]]:
    drop_cols = set([target] + drops)
    feat_cols = [c for c in df.columns if c not in drop_cols]
    num_cols = [c for c in feat_cols if pd.api.types.is_numeric_dtype(df[c])]
    cat_cols = [c for c in feat_cols if c not in num_cols]
    return feat_cols, num_cols, cat_cols

def make_preprocessor(num_cols, cat_cols):
    return ColumnTransformer(
        transformers=[
            ("num","passthrough", num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_cols)
        ]
    )

def threshold_for_topM(proba: np.ndarray, M: int) -> float:
    if M <= 0: return 1.0
    M = min(M, len(proba))
    return float(np.partition(proba, -M)[-M]) if M < len(proba) else 0.0

def kpi_block(g: pd.DataFrame) -> pd.Series:
    p = g["y_proba"].clip(0,1)
    N = len(p)
    C = float(p.sum())
    var = float((p*(1-p)).sum())
    se  = var**0.5
    return pd.Series({
        "N": N,
        "Esperados": C,
        "IC95_inf": C - 1.96*se,
        "IC95_sup": C + 1.96*se,
        "Tasa_Esp": C/N if N else np.nan,
        "Tasa_IC95_inf": (C - 1.96*se)/N if N else np.nan,
        "Tasa_IC95_sup": (C + 1.96*se)/N if N else np.nan
    })

5) EXTRACCIÓN Y LABELING

In [19]:
print(">> Extrayendo datos...")
df = pd.read_sql(SQL_EXTRACT, engine)
if df.empty:
    raise SystemExit("La consulta devolvió 0 filas.")

# Normaliza periodos y construye orden
df[APP.col_periodo] = norm_periodo(df[APP.col_periodo])
df[APP.col_periodo_paga] = norm_periodo(df[APP.col_periodo_paga])
df[APP.col_periodo_orden] = df[APP.col_periodo].map(ORDEN_MAP).astype("Int64")

# Etiqueta (y_rematricula): 1 si ESTADOACTUAL = 'YA PAGO' y Periodo_Paga es un periodo
# posterior al COD_PERIODO (según ORDEN_MAP). Evita fuga usando sólo info histórica.
def compute_label(row):
    est = str(row.get(APP.col_estado_actual, "")).upper()
    cp  = row.get(APP.col_periodo, None)
    pp  = row.get(APP.col_periodo_paga, None)
    if est == "YA PAGO":
        i_cp = ORDEN_MAP.get(str(cp), None)
        i_pp = ORDEN_MAP.get(str(pp), None)
        if i_cp is not None and i_pp is not None:
            return int(i_pp > i_cp)  # pago en periodo posterior => rematrícula
    return 0

df[APP.col_target] = df.apply(compute_label, axis=1).astype(int)

# Depura filas sin orden
df = df[df[APP.col_periodo_orden].notna()].copy()
df[APP.col_periodo_orden] = df[APP.col_periodo_orden].astype(int)

>> Extrayendo datos...


6) SPLIT TEMPORAL

In [20]:
v_max = int(df[APP.col_periodo_orden].max())
df_train = df[df[APP.col_periodo_orden] < v_max].copy()
df_valid = df[df[APP.col_periodo_orden] == v_max].copy()

if df_train.empty or df_valid.empty:
    raise ValueError("Split temporal inválido. Revisa 'periodo_orden' construido desde COD_PERIODO.")

print(f">> Periodo VALID = {v_max} | train={len(df_train):,} | valid={len(df_valid):,}")

>> Periodo VALID = 15 | train=249,573 | valid=773


7) PIPELINE GPU + CALIBRACIÓN

In [ ]:
drop_extra = [APP.col_periodo, APP.col_periodo_orden, APP.col_id, APP.col_periodo_paga]
feat_cols, num_cols, cat_cols = infer_feature_sets(df_train, APP.col_target, drop_extra)

pre = make_preprocessor(num_cols, cat_cols)

y_tr = df_train[APP.col_target].astype(int).to_numpy()
X_tr = df_train[feat_cols]
y_va = df_valid[APP.col_target].astype(int).to_numpy()
X_va = df_valid[feat_cols]

# Entrenamos un pipeline base (prep + XGBClassifier GPU)
xgb_gpu = build_gpu_xgb(pd.Series(y_tr))
pipe = Pipeline([("prep", pre), ("clf", xgb_gpu)])

# Predicciones OOF para calibración isotónica respetando tiempo
tscv = TimeSeriesSplit(n_splits=3)  # puedes subir a 5 si hay suficientes periodos
proba_oof = np.zeros(len(X_tr), dtype=float)

for tr_idx, te_idx in tscv.split(X_tr, y_tr):
    X_tr_fold = X_tr.iloc[tr_idx]
    y_tr_fold = y_tr[tr_idx]
    X_te_fold = X_tr.iloc[te_idx]

    pipe_fold = Pipeline([("prep", pre), ("clf", build_gpu_xgb(pd.Series(y_tr_fold)))])
    pipe_fold.fit(X_tr_fold, y_tr_fold)

    proba_oof[te_idx] = pipe_fold.predict_proba(X_te_fold)[:, 1]

# Ajuste isotónico con OOF
iso = IsotonicRegression(out_of_bounds="clip", y_min=0.0, y_max=1.0)
iso.fit(proba_oof, y_tr)

# Reentrenamos el pipeline final con TODO el train
pipe.fit(X_tr, y_tr)

# Probabilidades calibradas en VALID
proba_valid_raw = pipe.predict_proba(X_va)[:, 1]
y_proba_valid = iso.predict(proba_valid_raw)

# Métricas
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
metrics = {
    "AUC": float(roc_auc_score(y_va, y_proba_valid)),
    "AP": float(average_precision_score(y_va, y_proba_valid)),
    "Brier": float(brier_score_loss(y_va, y_proba_valid)),
}
print(f">> Métricas VALID: AUC={metrics['AUC']:.4f} | AP={metrics['AP']:.4f} | Brier={metrics['Brier']:.5f}")

C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [15:09:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [15:09:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local

>> Métricas VALID: AUC=0.9848 | AP=0.9243 | Brier=0.05992


C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [15:09:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


8) SCORING + KPI + UMBRAL

In [22]:
os.makedirs(APP.out_dir, exist_ok=True)
scoring = df_valid[[APP.col_id, APP.seg_programa, APP.seg_sede]].copy()
scoring["y_proba"] = y_proba_valid

out_scoring_path = os.path.join(APP.out_dir, APP.out_scoring)
scoring.to_csv(out_scoring_path, index=False, encoding="utf-8-sig")

kpi_global = kpi_block(scoring)
kpi_prog = scoring.groupby(APP.seg_programa, dropna=False).apply(kpi_block).reset_index()
kpi_sede = scoring.groupby(APP.seg_sede, dropna=False).apply(kpi_block).reset_index()

thr = threshold_for_topM(scoring["y_proba"].values, APP.topM)
scoring["gestion"] = (scoring["y_proba"] >= thr).astype(int)

# Validación del conteo si tienes etiqueta real en VALID
reales = int(df_valid[APP.col_target].sum())
esperados = float(scoring["y_proba"].sum())
abs_err = abs(esperados - reales)

print("\n=== KPI Global (conteo/tasa con IC95%) ===")
print(kpi_global)
print(f"\nUmbral por capacidad Top-M={APP.topM}: {thr:.6f}")
print(f"\nValidación conteo: Reales={reales:,} | Esperados={esperados:,.2f} | AbsErr={abs_err:,.2f}")

print("\nEjemplo KPI por Programa (top 10):")
print(kpi_prog.sort_values('Esperados', ascending=False).head(10))
print("\nEjemplo KPI por Sede (top 10):")
print(kpi_sede.sort_values('Esperados', ascending=False).head(10))

print(f"\n📄 Scoring guardado en: {out_scoring_path}")


=== KPI Global (conteo/tasa con IC95%) ===
N                773.000000
Esperados        359.507408
IC95_inf         339.969788
IC95_sup         379.045028
Tasa_Esp           0.465081
Tasa_IC95_inf      0.439806
Tasa_IC95_sup      0.490356
dtype: float64

Umbral por capacidad Top-M=1500: 0.000000

Validación conteo: Reales=214 | Esperados=359.51 | AbsErr=145.51

Ejemplo KPI por Programa (top 10):
                                          Programa      N   Esperados  \
12                          INGENIERIA DE SISTEMAS  207.0  105.893575   
0                       ADMINISTRACION DE EMPRESAS  121.0   54.116114   
11                                  DISEÑO GRAFICO  112.0   53.473567   
10                                 DISEÑO DE MODAS   71.0   32.981706   
8   DIRECCION Y PRODUCCION DE MEDIOS AUDIOVISUALES   53.0   28.679187   
5                              COMUNICACION SOCIAL   44.0   22.719698   
3             ADMINISTRACION EN SERVICIOS DE SALUD   47.0   16.598234   
13              

C:\Users\maria_castrob\AppData\Local\Temp\ipykernel_5236\2391740491.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_prog = scoring.groupby(APP.seg_programa, dropna=False).apply(kpi_block).reset_index()
C:\Users\maria_castrob\AppData\Local\Temp\ipykernel_5236\2391740491.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_sede = scoring.groupby(APP.seg_sede, dropna=False).apply(kpi_block).reset_index()


In [ ]:
def logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

# OOF con TimeSeriesSplit (5 folds)
tscv = TimeSeriesSplit(n_splits=5)

proba_oof = np.zeros(len(X_tr), dtype=float)
for tr_idx, te_idx in tscv.split(X_tr, y_tr):
    pipe_fold = Pipeline([("prep", pre), ("clf", build_gpu_xgb(pd.Series(y_tr[tr_idx])))])
    pipe_fold.fit(X_tr.iloc[tr_idx], y_tr[tr_idx])
    proba_oof[te_idx] = pipe_fold.predict_proba(X_tr.iloc[te_idx])[:,1]

# Platt: logística sobre logit(proba) 
X_platt = logit_clip(proba_oof).reshape(-1,1)
platt = LogisticRegression(solver="lbfgs")
platt.fit(X_platt, y_tr)

# Reentrena pipeline final en TODO el train
pipe.fit(X_tr, y_tr)

# Probabilidades calibradas en VALID
proba_valid_raw = pipe.predict_proba(X_va)[:,1]
y_proba_valid   = platt.predict_proba(logit_clip(proba_valid_raw).reshape(-1,1))[:,1]

print("AUC base:", roc_auc_score(y_va, proba_valid_raw))
print("AUC cal :", roc_auc_score(y_va, y_proba_valid))
print("AP cal  :", average_precision_score(y_va, y_proba_valid))
print("Brier   :", brier_score_loss(y_va, y_proba_valid))



AUC base: 0.9881756756756757
AUC cal : 0.9881756756756757
AP cal  : 0.8965298806565636
Brier   : 0.035543848640486465


In [ ]:

df_val_diag = pd.DataFrame({
    "y": y_va,
    "p_raw": proba_valid_raw,
    "p_cal": y_proba_valid
})

# Reliability (cuantiles)
prob_true, prob_pred = calibration_curve(df_val_diag["y"], df_val_diag["p_cal"], n_bins=10, strategy="quantile")
print("Calibración (quantiles) - pred vs true:")
print(pd.DataFrame({"pred_bin_mean": prob_pred, "obs_rate": prob_true}))

# Deciles: esperado vs observado
df_val_diag["decile"] = pd.qcut(df_val_diag["p_cal"], 10, labels=False, duplicates="drop")
tab = df_val_diag.groupby("decile").agg(
    N=("y","size"),
    Esperados=("p_cal","sum"),
    Reales=("y","sum"),
    p_med=("p_cal","mean")
).reset_index().sort_values("decile", ascending=False)
tab["Error"] = tab["Esperados"] - tab["Reales"]
print(tab)



Calibración (quantiles) - pred vs true:
   pred_bin_mean  obs_rate
0       0.103509  0.000000
1       0.126545  0.000000
2       0.141563  0.000000
3       0.153900  0.000000
4       0.164612  0.000000
5       0.176528  0.000000
6       0.189992  0.000000
7       0.592025  0.493333
8       0.951257  0.922078
9       0.958179  0.935897
   decile   N  Esperados  Reales     p_med      Error
9       9  78  74.737990      73  0.958179   1.737990
8       8  77  73.246761      71  0.951257   2.246761
7       7  75  44.401886      37  0.592025   7.401886
6       6  76  14.439362       0  0.189992  14.439362
5       5  80  14.122235       0  0.176528  14.122235
4       4  78  12.839735       0  0.164612  12.839735
3       3  77  11.850324       0  0.153900  11.850324
2       2  77  10.900348       0  0.141563  10.900348
1       1  77   9.743966       0  0.126545   9.743966
0       0  78   8.073717       0  0.103509   8.073717


In [ ]:
# Factor de conteo (s) para alinear suma de probas con reales
s = df_val_diag["y"].sum() / df_val_diag["p_cal"].sum()
s = float(np.clip(s, 0.6, 1.4))  # evita cambios extremos
df_val_diag["p_adj"] = np.clip(df_val_diag["p_cal"] * s, 0, 1)

# Recalcula KPI con p_adj
p = df_val_diag["p_adj"].values
N = len(p)
Esperados = p.sum()
Var = (p*(1-p)).sum()
IC = (Esperados - 1.96*np.sqrt(Var), Esperados + 1.96*np.sqrt(Var))
print(f"Conteo ajustado: {Esperados:.2f} | IC95%: [{IC[0]:.1f}, {IC[1]:.1f}] | Reales={df_val_diag['y'].sum()}")


Conteo ajustado: 181.00 | IC95%: [161.6, 200.4] | Reales=181


In [ ]:
M = 250  # ajusta a tu capacidad
thr = np.partition(df_val_diag["p_adj"].values, -M)[-M]
df_val_diag["gestionar"] = (df_val_diag["p_adj"] >= thr).astype(int)

# Precisión@M (cuántos aciertos entre los M priorizados)
topM = df_val_diag.sort_values("p_adj", ascending=False).head(M)
precision_at_M = topM["y"].mean()
print(f"Precision@{M}: {precision_at_M:.3f}")


Precision@250: 0.724


In [ ]:
# COUNT CALIBRATION (guardarraíl) + KPI 
import numpy as np
import pandas as pd
from joblib import dump

# df_valid: contiene y_va y columnas identificadoras
# y_proba_valid: probas calibradas (isotónica o Platt)
# proba_valid_raw: probas sin calibrar (solo si quieres comparar)
# Función kpi_block ya definida antes

df_val_diag = pd.DataFrame({
    "y": y_va.astype(int),
    "p_cal": y_proba_valid
})

# Factor de conteo s (ajusta suma de probas ~ reales)
reales = df_val_diag["y"].sum()
esperados = df_val_diag["p_cal"].sum()
s = float(np.clip(reales / max(esperados, 1e-9), 0.6, 1.4))   # límites sanos
print(f"Factor s = {s:.3f} | Reales={reales} | Esperados_cal={esperados:.2f}")

# Probabilidad ajustada y KPI global
df_val_diag["p_adj"] = np.clip(df_val_diag["p_cal"] * s, 0, 1)

N = len(df_val_diag)
C = df_val_diag["p_adj"].sum()
var = (df_val_diag["p_adj"] * (1 - df_val_diag["p_adj"])).sum()
se = var ** 0.5
print("\n=== KPI Global (ajustado) ===")
print(pd.Series({
    "N": N,
    "Esperados_adj": C,
    "IC95_inf": C - 1.96 * se,
    "IC95_sup": C + 1.96 * se,
    "Tasa_adj": C / N,
    "Tasa_IC95_inf": (C - 1.96 * se) / N,
    "Tasa_IC95_sup": (C + 1.96 * se) / N
}))

# KPI por Programa / Sede con p_adj
scoring_adj = df_valid[[APP.col_id, APP.seg_programa, APP.seg_sede]].copy()
scoring_adj["y_proba"] = df_val_diag["p_adj"].values

kpi_prog_adj = scoring_adj.groupby(APP.seg_programa, dropna=False).apply(kpi_block).reset_index()
kpi_sede_adj = scoring_adj.groupby(APP.seg_sede, dropna=False).apply(kpi_block).reset_index()

print("\nTop Programas por Esperados_adj")
print(kpi_prog_adj.sort_values("Esperados", ascending=False).head(10))
print("\nTop Sedes por Esperados_adj")
print(kpi_sede_adj.sort_values("Esperados", ascending=False).head(10))

# Top-M para gestión (usa p_adj)
APP.topM = min(250, len(scoring_adj))   # ejemplo de capacidad
thr_adj = np.partition(scoring_adj["y_proba"].values, -APP.topM)[-APP.topM]
scoring_adj["gestion"] = (scoring_adj["y_proba"] >= thr_adj).astype(int)

print(f"\nUmbral ajustado Top-M={APP.topM}: {thr_adj:.6f}")
print("Precision@M:", scoring_adj.sort_values("y_proba", ascending=False).head(APP.topM)["gestion"].mean())

# Export
os.makedirs(APP.out_dir, exist_ok=True)
out_scoring_path = os.path.join(APP.out_dir, "scoring_valid_ajustado.csv")
scoring_adj.to_csv(out_scoring_path, index=False, encoding="utf-8-sig")
print(f"\n📄 Scoring ajustado guardado en: {out_scoring_path}")

# Persistir artefactos para producción:
art_dir = os.path.join(APP.out_dir, "artefactos_modelo")
os.makedirs(art_dir, exist_ok=True)
dump(pipe, os.path.join(art_dir, "pipe_xgb_gpu.joblib"))
try:
    dump(iso, os.path.join(art_dir, "calibrador_isotonic.joblib"))
except NameError:
    try:
        dump(platt, os.path.join(art_dir, "calibrador_platt.joblib"))
    except NameError:
        pass
pd.Series({"s": s}).to_json(os.path.join(art_dir, "count_factor.json"))
print(f"✔ Artefactos guardados en: {art_dir}")


Factor s = 0.660 | Reales=181 | Esperados_cal=274.36

=== KPI Global (ajustado) ===
N                773.000000
Esperados_adj    181.000000
IC95_inf         161.580706
IC95_sup         200.419294
Tasa_adj           0.234153
Tasa_IC95_inf      0.209031
Tasa_IC95_sup      0.259275
dtype: float64

Top Programas por Esperados_adj
                                          Programa      N  Esperados  \
12                          INGENIERIA DE SISTEMAS  207.0  58.524432   
11                                  DISEÑO GRAFICO  112.0  27.408965   
0                       ADMINISTRACION DE EMPRESAS  121.0  26.256145   
10                                 DISEÑO DE MODAS   71.0  16.418092   
8   DIRECCION Y PRODUCCION DE MEDIOS AUDIOVISUALES   53.0  13.042554   
5                              COMUNICACION SOCIAL   44.0   9.735335   
3             ADMINISTRACION EN SERVICIOS DE SALUD   47.0   9.244499   
13                          INGENIERIA ELECTRONICA   36.0   8.737098   
7                       

In [ ]:
# precisión en los M priorizados (usa la etiqueta real)
topM = pd.DataFrame({
    "y": y_va.astype(int),
    "p_adj": scoring_adj["y_proba"].values
}).sort_values("p_adj", ascending=False).head(APP.topM)

precision_at_M = topM["y"].mean()
recall_at_M    = topM["y"].sum() / y_va.sum()
lift_at_M      = precision_at_M / y_va.mean()

print(f"Precision@{APP.topM}: {precision_at_M:.3f}")
print(f"Recall@{APP.topM}:    {recall_at_M:.3f}")
print(f"Lift@{APP.topM}:      {lift_at_M:.2f}x")


Precision@250: 0.724
Recall@250:    1.000
Lift@250:      3.09x


In [ ]:
def tabla_control(seg):
    tmp = scoring_adj[[seg]].copy()
    tmp["Esperados_adj"] = scoring_adj["y_proba"].values
    tmp["Reales"]        = y_va
    g = tmp.groupby(seg, dropna=False).agg(
        N=("Reales","size"),
        Esperados=("Esperados_adj","sum"),
        Reales=("Reales","sum")
    ).reset_index()
    g["Error"] = g["Esperados"] - g["Reales"]
    g["AbsErr"] = g["Error"].abs()
    return g.sort_values("AbsErr", ascending=False)

ctrl_prog = tabla_control(APP.seg_programa).head(15)
ctrl_sede = tabla_control(APP.seg_sede).head(15)
print(ctrl_prog.head(10))
print(ctrl_sede.head(10))


                                          Programa    N  Esperados  Reales  \
12                          INGENIERIA DE SISTEMAS  207  58.524432      68   
2                         ADMINISTRACION DEPORTIVA   28   3.072760       1   
14                        NEGOCIOS INTERNACIONALES    9   1.384089       0   
0                       ADMINISTRACION DE EMPRESAS  121  26.256145      25   
3             ADMINISTRACION EN SERVICIOS DE SALUD   47   9.244499       8   
8   DIRECCION Y PRODUCCION DE MEDIOS AUDIOVISUALES   53  13.042554      12   
1            ADMINISTRACION DE LA SEGURIDAD SOCIAL    8   0.918335       0   
13                          INGENIERIA ELECTRONICA   36   8.737098       8   
4              ADMINISTRACION TURISTICA Y HOTELERA    7   0.611940       0   
10                                 DISEÑO DE MODAS   71  16.418092      17   

       Error    AbsErr  
12 -9.475568  9.475568  
2   2.072760  2.072760  
14  1.384089  1.384089  
0   1.256145  1.256145  
3   1.244499  1.

In [23]:
def kpi_block_safe(g: pd.DataFrame) -> pd.Series:
    p = g["y_proba"].clip(0,1)
    N = len(p)
    C = float(p.sum())
    var = float((p*(1-p)).sum())
    se  = var**0.5
    ic_inf = max(0.0, C - 1.96*se)
    ic_sup = min(N,   C + 1.96*se)
    return pd.Series({
        "N": N, "Esperados": C,
        "IC95_inf": ic_inf, "IC95_sup": ic_sup,
        "Tasa_Esp": C/N if N else np.nan,
        "Tasa_IC95_inf": ic_inf/N if N else np.nan,
        "Tasa_IC95_sup": ic_sup/N if N else np.nan
    })


modelo v2 

In [ ]:
import os, numpy as np, pandas as pd
from dataclasses import dataclass
from typing import List, Tuple
from joblib import dump

# ML 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from xgboost import XGBClassifier

# SQL
from sqlalchemy import create_engine, text

In [ ]:
import os
# Conexión SQL 
host = os.environ.get('SQL_SERVER','')
port = os.environ.get('SQL_PORT','1433')
database = os.environ.get('SQL_DB','')
driver = 'ODBC Driver 17 for SQL Server'

engine_url = (
    f"mssql+pyodbc://@{host}:{port}/{database}"
    f"?driver={driver.replace(' ', '+')}"
    f"&trusted_connection=yes&TrustServerCertificate=yes"
)
engine = create_engine(engine_url, fast_executemany=True)

# App 
@dataclass
class APP:
    col_id: str = "num_identificacion"
    col_periodo: str = "COD_PERIODO"
    col_periodo_paga: str = "Periodo_Paga"
    col_estado_actual: str = "ESTADOACTUAL"
    col_periodo_orden: str = "periodo_orden"
    col_target: str = "y_rematricula"      # se construye
    seg_programa: str = "Programa"
    seg_sede: str = "sede"
    out_dir: str = r"C:\visual\rematricula_out"
    scoring_file: str = "scoring_periodo.csv"
    artefactos_dir: str = "artefactos_modelo"
    topM: int = 300                      

APP = APP()

# Ordenamiento de periodos (ajustar si ingresan nuevos)
PERIODO_ORDEN = [
    "23V04","2023B","23V05","23V06",
    "24V01","2024A","24V02","24V03","24P02","24V04","2024B","24V05","24V06",
    "25V02","2025B","25V05","2025D"
]
ORDEN_MAP = {p.upper(): i for i, p in enumerate(PERIODO_ORDEN, start=1)}



In [ ]:
SQL_EXTRACT = r"""
select
     [num_identificacion]
    ,[cod_pensum]
    ,[COD_PERIODO]
    ,[SEMESTRE]
    ,[ESTADO_ALUMNO]
    ,[Programa]
    ,[promedio]
    ,[CreditosAprobados]
    ,[CreditosTotales]
    ,[Creditos_Pendientes]
    ,[ULT_PER_PAGO]
    ,[Saldo]
    ,[Dupl]
    ,[Asignaturas_Aprobadas]
    ,[Asignaturas_Reprobadas]
    ,[Promedio_Per_Solicitado]
    ,case
        when [Status] = 'Retiro' AND ESTADOACTUAL = 'YA PAGO' THEN 'Posible Perdida Academica'
        when [Status] = 'Posible Egresado' AND ESTADOACTUAL = 'YA PAGO' THEN 'Posible Rematricula'
        else [Status] end as Status,
    NUEVO,
    UBICACION,
    sede,
    ESTADOACTUAL,
    Periodo_Paga
FROM
(
    SELECT
        a.[num_identificacion], a.[cod_pensum], a.[COD_PERIODO], a.[SEMESTRE],
        a.[ESTADO_ALUMNO], a.[Programa], a.[promedio],
        a.[CreditosAprobados], a.[CreditosTotales], a.[Creditos_Pendientes],
        a.[ULT_PER_PAGO], a.[Saldo], a.[Dupl],
        a.[Asignaturas_Aprobadas], a.[Asignaturas_Reprobadas], a.[Promedio_Per_Solicitado],
        a.[Status],
        t2.NUEVO, t2.UBICACION, t2.sede,
        case
            when t1.PERIODO is null then 'NO HA PAGADO'
            when t1.PERIODO = a.COD_PERIODO THEN 'NO HA PAGADO'
            else 'YA PAGO'
        end AS ESTADOACTUAL,
        case
            when t1.PERIODO is null then 'NO HA PAGADO'
            else t1.PERIODO
        end as Periodo_Paga
    from CUN_REPOSITORIO.GGALINDO.REMATRICULASNEW a
    left join (
        select DISTINCT Documento_Estudiante_zoho, max(PERIODO) as PERIODO
        from CUN_REPOSITORIO.dbo.Recibos_Contact2_Sin_Zoho
        where PERIODO in ('24V01','2024A','24V02','24V03','24P02','24V04','2024B','24V05','24V06','25v02','2025B','25v05','2025D')
          AND ESTADO_PAGO = 'PAGO'
          AND NCR_ANULACION = 0
        GROUP BY Documento_Estudiante_zoho
    ) t1 on a.num_identificacion = t1.Documento_Estudiante_zoho
    left join (
        select DISTINCT Documento_Estudiante_zoho, PERIODO, max(NUEVO) as NUEVO, UBICACION, sede
        from CUN_REPOSITORIO.dbo.Recibos_Contact2_Sin_Zoho
        where PERIODO in ('23V04','2023B','23V05','23V06','24V01','2024A','24V05','24V06','25v02','2025B')
          --AND ESTADO_PAGO = 'PAGO'
          AND NCR_ANULACION = 0
        GROUP BY Documento_Estudiante_zoho, PERIODO, UBICACION, sede
    ) t2 on a.num_identificacion = t2.Documento_Estudiante_zoho
       and a.COD_PERIODO = t2.PERIODO
) BASE
"""

def norm_periodo(s: pd.Series) -> pd.Series:
    return s.astype(str).str.upper().str.strip()

with engine.connect() as conn:
    ver = conn.execute(text("SELECT @@VERSION")).scalar()
    dbn = conn.execute(text("SELECT DB_NAME()")).scalar()
    print("✅ Conectado a:", dbn)
    print("🖥️ Servidor:", ver.splitlines()[0])

print(">> Extrayendo datos...")
df = pd.read_sql(SQL_EXTRACT, engine)
if df.empty:
    raise SystemExit("La consulta devolvió 0 filas.")

# Periodos y orden
df[APP.col_periodo] = norm_periodo(df[APP.col_periodo])
df[APP.col_periodo_paga] = norm_periodo(df[APP.col_periodo_paga])
df[APP.col_periodo_orden] = df[APP.col_periodo].map(ORDEN_MAP).astype("Int64")
df = df[df[APP.col_periodo_orden].notna()].copy()
df[APP.col_periodo_orden] = df[APP.col_periodo_orden].astype(int)

# SPLIT TRAIN / VALID

# Tomar automáticamente el último periodo disponible como validación
v_max = df[APP.col_periodo_orden].max()
periodo_obj = df.loc[df[APP.col_periodo_orden] == v_max, APP.col_periodo].mode().iat[0]

# Split en train y valid
df_train = df[df[APP.col_periodo_orden] < v_max].copy()
df_valid = df[df[APP.col_periodo_orden] == v_max].copy()

print(f">> Periodo objetivo: {periodo_obj} (orden={v_max})")
print("Registros totales:", len(df))
print("Registros train:", len(df_train))
print("Registros valid:", len(df_valid))



✅ Conectado a: CUN_REPOSITORIO
🖥️ Servidor: Microsoft SQL Server 2022 (RTM-GDR) (KB5035432) - 16.0.1115.1 (X64) 
>> Extrayendo datos...
>> Periodo objetivo: 2025B (orden=15)
Registros totales: 280876
Registros train: 279559
Registros valid: 1317


In [ ]:
# LABEL (sin fuga)
def compute_label(row):
    est = str(row.get(APP.col_estado_actual, "")).upper()
    cp  = str(row.get(APP.col_periodo, ""))
    pp  = str(row.get(APP.col_periodo_paga, ""))
    i_cp = ORDEN_MAP.get(cp)
    i_pp = ORDEN_MAP.get(pp)
    if est == "YA PAGO" and (i_cp is not None) and (i_pp is not None):
        return int(i_pp > i_cp)
    return 0

df[APP.col_target] = df.apply(compute_label, axis=1).astype(int)

In [ ]:
# SPLIT temporal
df_train = df[df[APP.col_periodo_orden] < v_max].copy()
df_valid = df[df[APP.col_periodo_orden] == v_max].copy()
print(f">> Train={len(df_train):,}  | Valid (periodo {periodo_obj})={len(df_valid):,}")


>> Train=279,559  | Valid (periodo 2025B)=1,317


In [ ]:
# Pipeline + Platt OOF + Count Calibration
def infer_feature_sets(df_: pd.DataFrame, target: str, drops: List[str]) -> Tuple[List[str], List[str], List[str]]:
    drop_cols = set([target] + drops)
    feat = [c for c in df_.columns if c not in drop_cols]
    num  = [c for c in feat if pd.api.types.is_numeric_dtype(df_[c])]
    cat  = [c for c in feat if c not in num]
    return feat, num, cat

def make_preprocessor(num_cols, cat_cols):
    return ColumnTransformer(
        transformers=[
            ("num", "passthrough", num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_cols),
        ]
    )

def build_gpu_xgb(y_train: pd.Series) -> XGBClassifier:
    pos = int((y_train==1).sum()); neg = int((y_train==0).sum())
    spw = max(neg / max(pos,1), 1.0)
    n_jobs = max((os.cpu_count() or 8) - 2, 4)
    return XGBClassifier(
        n_estimators=1200, max_depth=6, learning_rate=0.03,
        subsample=0.8, colsample_bytree=0.8, min_child_weight=4, reg_lambda=1.2,
        objective="binary:logistic", eval_metric="auc",
        tree_method="gpu_hist", predictor="gpu_predictor", gpu_id=0,
        scale_pos_weight=spw, n_jobs=n_jobs
    )

def logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

drop_extra = [APP.col_id, APP.col_periodo, APP.col_periodo_paga, APP.col_periodo_orden]
feat_cols, num_cols, cat_cols = infer_feature_sets(df_train, APP.col_target, drop_extra)
pre = make_preprocessor(num_cols, cat_cols)

X_tr = df_train[feat_cols]; y_tr = df_train[APP.col_target].astype(int).to_numpy()
X_va = df_valid[feat_cols]; y_va = df_valid[APP.col_target].astype(int).to_numpy()

# OOF para Platt (5 folds temporales)
tscv = TimeSeriesSplit(n_splits=5)
proba_oof = np.zeros(len(X_tr))
for tr_idx, te_idx in tscv.split(X_tr, y_tr):
    pipe_fold = Pipeline([("prep", pre), ("clf", build_gpu_xgb(pd.Series(y_tr[tr_idx])))])
    pipe_fold.fit(X_tr.iloc[tr_idx], y_tr[tr_idx])
    proba_oof[te_idx] = pipe_fold.predict_proba(X_tr.iloc[te_idx])[:,1]

platt = LogisticRegression(solver="lbfgs")
platt.fit(logit_clip(proba_oof).reshape(-1,1), y_tr)

# Modelo final en todo el train
pipe = Pipeline([("prep", pre), ("clf", build_gpu_xgb(pd.Series(y_tr)))])
pipe.fit(X_tr, y_tr)

# Valid sin y con calibración
proba_valid_raw = pipe.predict_proba(X_va)[:,1]
p_cal_valid = platt.predict_proba(logit_clip(proba_valid_raw).reshape(-1,1))[:,1]

auc_raw = roc_auc_score(y_va, proba_valid_raw)
auc_cal = roc_auc_score(y_va, p_cal_valid)
brier   = brier_score_loss(y_va, p_cal_valid)
ap      = average_precision_score(y_va, p_cal_valid)
print(f">> VALID – AUC raw={auc_raw:.4f} | AUC cal={auc_cal:.4f} | AP={ap:.4f} | Brier={brier:.5f}")

# Count calibration s
reales_valid = y_va.sum()
esperados_cal = p_cal_valid.sum()
s = float(np.clip(reales_valid / max(esperados_cal,1e-9), 0.6, 1.4))
print(f">> Count-calibration: s={s:.3f} | Reales={reales_valid} | Esperados_cal={esperados_cal:.2f}")


C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [16:54:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [16:54:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local

>> VALID – AUC raw=0.9872 | AUC cal=0.9872 | AP=0.9023 | Brier=0.03761
>> Count-calibration: s=0.643 | Reales=309 | Esperados_cal=480.55


C:\Users\maria_castrob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [16:55:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


In [ ]:
# 5) Proyección en el universo del periodo objetivo

X_full = df_valid[feat_cols]
p_raw_full = pipe.predict_proba(X_full)[:,1]
p_cal_full = platt.predict_proba(logit_clip(p_raw_full).reshape(-1,1))[:,1]
p_adj_full = np.clip(p_cal_full * s, 0, 1)

# Pagados ya
df_valid["ya_pago"] = (df_valid[APP.col_estado_actual].str.upper() == "YA PAGO").astype(int)
pagados = int(df_valid["ya_pago"].sum())

# Esperados adicionales (solo NO pagados)
esperados_no_pag = float(p_adj_full[df_valid["ya_pago"].values==0].sum())
total_proyectado = pagados + esperados_no_pag

print("\n===== PROYECCIÓN PERIODO OBJETIVO =====")
print(f"Periodo: {periodo_obj}")
print(f"N total (universo): {len(df_valid):,}")
print(f"Pagados actuales:   {pagados:,}")
print(f"Esperados adicionales (modelo): {esperados_no_pag:,.0f}")
print(f"TOTAL proyectado de rematriculados: {total_proyectado:,.0f}")


===== PROYECCIÓN PERIODO OBJETIVO =====
Periodo: 2025B
N total (universo): 1,317
Pagados actuales:   331
Esperados adicionales (modelo): 107
TOTAL proyectado de rematriculados: 438


In [ ]:
# 6) KPI por segmentos + Export
# =========================
def kpi_block(g: pd.DataFrame) -> pd.Series:
    p = g["y_proba"].clip(0,1)
    N = len(p); C = float(p.sum())
    var = float((p*(1-p)).sum()); se = var**0.5
    ic_inf = max(0.0, C - 1.96*se); ic_sup = min(N, C + 1.96*se)
    return pd.Series({
        "N": N, "Esperados": C, "IC95_inf": ic_inf, "IC95_sup": ic_sup,
        "Tasa_Esp": C/N if N else np.nan,
        "Tasa_IC95_inf": ic_inf/N if N else np.nan,
        "Tasa_IC95_sup": ic_sup/N if N else np.nan
    })

# Scoring para no pagados (gestión) + para KPI global
scoring = df_valid[[APP.col_id, APP.seg_programa, APP.seg_sede]].copy()
scoring["ya_pago"] = df_valid["ya_pago"].values
scoring["y_proba"] = p_adj_full

# KPIs por Programa/Sede (sobre todo el universo del periodo)
kpi_prog = scoring.groupby(APP.seg_programa, dropna=False).apply(kpi_block).reset_index()
kpi_sede = scoring.groupby(APP.seg_sede, dropna=False).apply(kpi_block).reset_index()

# Top-M (solo sobre NO pagados)
pend = scoring[scoring["ya_pago"]==0].copy()
M = min(APP.topM, len(pend))
thr = np.partition(pend["y_proba"].values, -M)[-M] if M>0 else 1.0
pend["gestion"] = (pend["y_proba"] >= thr).astype(int)

# Export
os.makedirs(APP.out_dir, exist_ok=True)
scoring_path = os.path.join(APP.out_dir, APP.scoring_file)
kpi_prog_path = os.path.join(APP.out_dir, "kpi_programa.csv")
kpi_sede_path = os.path.join(APP.out_dir, "kpi_sede.csv")
scoring.to_csv(scoring_path, index=False, encoding="utf-8-sig")
kpi_prog.to_csv(kpi_prog_path, index=False, encoding="utf-8-sig")
kpi_sede.to_csv(kpi_sede_path, index=False, encoding="utf-8-sig")

print(f"\n📄 Scoring guardado: {scoring_path}")
print(f"📄 KPI Programa:     {kpi_prog_path}")
print(f"📄 KPI Sede:         {kpi_sede_path}")


📄 Scoring guardado: C:\visual\rematricula_out\scoring_periodo.csv
📄 KPI Programa:     C:\visual\rematricula_out\kpi_programa.csv
📄 KPI Sede:         C:\visual\rematricula_out\kpi_sede.csv


C:\Users\maria_castrob\AppData\Local\Temp\ipykernel_12748\1201174826.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_prog = scoring.groupby(APP.seg_programa, dropna=False).apply(kpi_block).reset_index()
C:\Users\maria_castrob\AppData\Local\Temp\ipykernel_12748\1201174826.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_sede = scoring.groupby(APP.seg_sede, dropna=False).apply(kpi_block).reset_index()


In [ ]:
# Artefactos
art_dir = os.path.join(APP.out_dir, APP.artefactos_dir)
os.makedirs(art_dir, exist_ok=True)
dump(pipe,  os.path.join(art_dir, "pipe_xgb_gpu.joblib"))
dump(platt, os.path.join(art_dir, "calibrador_platt.joblib"))
pd.Series({"s": float(s), "periodo_objetivo": periodo_obj}).to_json(os.path.join(art_dir, "count_factor.json"))
print(f"✔ Artefactos guardados en: {art_dir}")

✔ Artefactos guardados en: C:\visual\rematricula_out\artefactos_modelo


In [ ]:
for per in periodo_obj:
    df_check = df[df[APP.col_periodo] == per]
    print("N en", per, "=", len(df_check))
    print(df_check[APP.col_estado_actual].value_counts(dropna=False))
    print("----")


N en 2025B = 1317
ESTADOACTUAL
NO HA PAGADO    986
YA PAGO         331
Name: count, dtype: int64
----


In [ ]:
def resumen_periodo(df, periodo: str):
    df_check = df[df[APP.col_periodo] == periodo]
    print(f"N en {periodo} = {len(df_check)}")
    print(df_check[APP.col_estado_actual].value_counts(dropna=False))
    return df_check

# Ejemplo
resumen_periodo(df, "25V02")


N en 25V02 = 19913
ESTADOACTUAL
NO HA PAGADO    11652
YA PAGO          8261
Name: count, dtype: int64


,num_identificacion,cod_pensum,COD_PERIODO,SEMESTRE,ESTADO_ALUMNO,Programa,promedio,CreditosAprobados,CreditosTotales,Creditos_Pendientes,...,Asignaturas_Reprobadas,Promedio_Per_Solicitado,Status,NUEVO,UBICACION,sede,ESTADOACTUAL,Periodo_Paga,periodo_orden,y_rematricula
10,1012410741,1073,25V02,2.0,1-Activo,CONTADURIA PUBLICA,4.20,6.0,6.0,0.0,...,0.0,4.19750,Posible Egresado,ANTIGUO,Colombia - Cundinamarca - Silvania,BOGOTA CENTRO,NO HA PAGADO,25V02,14,0
11,1000615289,5016,25V02,1.0,1-Activo,DISEÑO GRAFICO,4.60,5.0,5.0,0.0,...,0.0,4.58375,Posible Egresado,NUEVO,Colombia - Bogot- DC,BOGOTA CENTRO,NO HA PAGADO,25V02,14,0
13,1056554181,1073,25V02,1.0,1-Activo,CONTADURIA PUBLICA,4.58,6.0,6.0,0.0,...,0.0,4.55625,Posible Rematricula,NUEVO,Colombia - Boyaca - Socha,None,YA PAGO,25V05,14,1
16,1000062359,1037,25V02,6.0,1-Activo,ADMINISTRACION DE EMPRESAS,3.71,6.0,6.0,0.0,...,2.0,3.71250,Posible Egresado,NUEVO,Colombia - Bogot- DC,None,NO HA PAGADO,25V02,14,0
19,1003825433,1037,25V02,2.0,1-Activo,ADMINISTRACION DE EMPRESAS,2.53,5.0,5.0,0.0,...,5.0,2.53625,Posible Perdida Academica,NUEVO,Colombia - Bogot- DC,None,NO HA PAGADO,25V02,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197179,1077723757,1073,25V02,5.0,1-Activo,CONTADURIA PUBLICA,3.74,6.0,6.0,0.0,...,1.0,3.72000,Posible Egresado,ANTIGUO,Colombia - Huila - Neiva,None,NO HA PAGADO,25V02,14,0
197378,1105790684,1110,25V02,1.0,1-Activo,INGENIERIA INDUSTRIAL,3.64,5.0,5.0,0.0,...,1.0,3.70000,Posible Rematricula,NUEVO,Colombia - Cundinamarca - Madrid,BOGOTA CENTRO,YA PAGO,25V05,14,1
197886,1001118213,1120,25V02,1.0,10-Anulado,PUBLICIDAD Y MERCADEO,NaN,NaN,NaN,NaN,...,0.0,4.12250,Posible Rematricula,NUEVO,Colombia - Bogot- DC,BOGOTA CENTRO,NO HA PAGADO,25V02,14,0
197889,1001118213,3032,25V02,1.0,1-Activo,COMUNICACION SOCIAL,4.15,5.0,5.0,0.0,...,0.0,4.12250,Posible Egresado,NUEVO,Colombia - Bogot- DC,BOGOTA CENTRO,NO HA PAGADO,25V02,14,0
